In [1]:
# !pip install transformers datasets torch langchain-community faiss-cpu sentence-transformers
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

In [2]:
import gradio as gr
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub

/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import yaml
import fitz
import torch
import gradio as gr
from PIL import Image
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from datasets import load_dataset
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

# Add Text to Chat History
def add_text(history, text):
    if not text:
        raise gr.Error('Enter text')
    history.append((text, ''))
    return history

# Create Prompt Template
def create_prompt_template():
    template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Answer professionally, to the best of your ability, and where appropriate, in a Computer Science educational context.
    Use the context and be specific as you can.
    Context: {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
    
    return PromptTemplate.from_template(QA_CHAIN_PROMPT)

# Load Embeddings
def load_embeddings(model_name):
    return HuggingFaceEmbeddings(model_name=model_name)

# Load Vector Database
def load_vectordb(documents, embeddings):
    return FAISS.from_documents(documents, embeddings)

# Load Tokenizer
def load_tokenizer(model_name):
    return AutoTokenizer.from_pretrained(model_name)

# Load Model
def load_model(model_name):
    return AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='auto',
        torch_dtype=torch.float32,
        token=True,
        load_in_8bit=False
    )

# Create Pipeline
def create_pipeline(model, tokenizer):
    pipe = pipeline(
        model=model,
        task='text-generation',
        tokenizer=tokenizer,
        max_new_tokens=200
    )
    return HuggingFacePipeline(pipeline=pipe)

# Create Conversational Retrieval Chain
def create_chain(llm, retriever, prompt):
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": prompt})
    return qa

# Process File
def process_file(file_name, model_name):
    prompt = create_prompt_template()
    loader = CSVLoader(file_name)
    documents = loader.load()  

    modelPath = "sentence-transformers/gtr-t5-base" # Use a t5 sentence transformer model that maps sentences & paragraphs to a 768 dimensional dense vector space
    model_kwargs = {'device':'cpu'}
    encode_kwargs = {'normalize_embeddings': True} # Normalizing embeddings can help improve similarity metrics by ensuring that embeddings magnitude does not affect the similarity scores

    # Initialize an instance of HuggingFaceEmbeddings

    embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    )

    db = FAISS.from_documents(d, embedding=embeddings)
    retriever = db.as_retriever()
    # vectordb = load_vectordb(documents, embeddings)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})
    
    chain = create_chain(llm, retriever, prompt)
    return chain, documents

# Generate Response
def generate_response(history, query, chain, documents, page):
    if not query:
        raise gr.Error(message='Submit a question')
    result = chain({"question": query, 'chat_history': history}, return_only_outputs=True)
    history.append((query, result["answer"]))
    page = list(result['source_documents'][0])[1][1]['page']
    return history, page

# Render File
def render_file(file, page):
    doc = fitz.open(file.name)
    page = doc[page]
    pix = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72))
    image = Image.frombytes('RGB', [pix.width, pix.height], pix.samples)
    return image

# Create Gradio Interface
def create_demo():
    with gr.Blocks(title= "RAG Chatbot Q&A",
        theme = "Soft"
        ) as demo:
        with gr.Column():
            with gr.Row():
                chat_history = gr.Chatbot(value=[], elem_id='chatbot', height=680)
                show_img = gr.Image(label='Overview', height=680)

        with gr.Row():
            with gr.Column(scale=0.60):
                text_input = gr.Textbox(
                    show_label=False,
                    placeholder="Type here to ask your PDF",
                container=False)

            with gr.Column(scale=0.20):
                submit_button = gr.Button('Send')

            with gr.Column(scale=0.20):
                uploaded_pdf = gr.UploadButton("📁 Upload PDF", file_types=[".pdf"])
                

        return demo, chat_history, show_img, text_input, submit_button, uploaded_pdf

# Set up event handlers
def set_event_handlers(demo, chat_history, show_img, text_input, submit_button, uploaded_pdf):
    with demo:
        # Event handler for uploading a PDF
        uploaded_pdf.upload(renderfile, inputs=[uploaded_pdf], outputs=[show_img])

        # Event handler for submitting text and generating response
        submit_button.click(add_text, inputs=[chat_history, text_input], outputs=[chat_history], queue=False).\
            success(generate_response, inputs=[chat_history, text_input, uploaded_pdf], outputs=[chat_history, text_input]).\
            success(render_file, inputs=[uploaded_pdf], outputs=[show_img])

# Launch the application
def launch_app(demo):
    demo.queue()
    demo.launch()

# Main execution flow
def main():
    demo, chat_history, show_img, text_input, submit_button, uploaded_pdf = create_demo()
    # pdf_chatbot = PDFChatBot()
    set_event_handlers(demo, chat_history, show_img, text_input, submit_button, uploaded_pdf)
    launch_app(demo)

# Execute the main function
main()


ModuleNotFoundError: No module named 'frontend'